### Sources 

https://baseballsavant.mlb.com/leaderboard

https://www.retrosheet.org/downloads/othercsvs.html

https://github.com/chadwickbureau/register/tree/master

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import matplotlib.ticker as ticker
import re
import glob 

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

In [ ]:
# Read files 
pitchers = pd.read_csv("datasets/raw/pitchers.csv")
batters = pd.read_csv("datasets/raw/batters.csv")
plays = pd.concat(map(pd.read_csv, glob.glob("datasets/raw/retrosheet/*plays.csv")), ignore_index=True) 

dtype_spec = {
    8: str, 
    9: str,  
    10: str 
}

register = pd.concat(
    map(lambda f: pd.read_csv(f, dtype=dtype_spec), 
        glob.glob("datasets/raw/chadwick_bureau_register/people-*.csv")),
    ignore_index=True
)


In [ ]:
# Extract ids for matching 
mlb_ids = np.unique(np.concatenate((pitchers["player_id"], batters["player_id"])))
print(f"Number of unique MLB ids: {len(mlb_ids)}")

retro_ids = np.unique(np.concatenate((plays["batter"], plays["pitcher"])).astype(str))
print(f"Number of unique retrosheet ids: {len(retro_ids)}")

player_id_map = register[(register['key_mlbam'].isin(mlb_ids) | register['key_retro'].isin(retro_ids))][['key_mlbam', 'key_retro']]

In [ ]:
pitchers = pd.merge(pitchers, player_id_map, left_on='player_id', right_on='key_mlbam', how='left')
batters = pd.merge(batters, player_id_map, left_on='player_id', right_on='key_mlbam', how='left')

In [ ]:
# Replace any NA for a pitch type with 0 
pitchers.loc[:, pitchers.columns.str.endswith("_formatted")] = pitchers.loc[:, pitchers.columns.str.endswith("_formatted")].fillna(0)
# Rename column to be pitch type percentage 
pitchers = pitchers.rename(columns=lambda col: col[2:-10] + "_percent" if col.endswith("_formatted") and col.startswith("n_") else col)

In [ ]:
pitchers_clean = pitchers[
    [
        "key_retro",
        "year",
        "k_percent",
        "bb_percent",
        "pitch_hand",
        "ff_percent",
        "sl_percent",
        "ch_percent",
        "cu_percent",
        "si_percent",
        "fc_percent",
        "fs_percent",
        "kn_percent",
        "st_percent",
        "sv_percent",
        "fo_percent",
        "sc_percent",
        "fastball_percent",
        "fastball_avg_speed",
    ]
]

# Drop any pitchers that don't have an average fastball velo (these are position players who pitched)
pitchers_clean = pitchers_clean.dropna(subset=['fastball_avg_speed'])

# Set the index to be the key and year 
pitchers_clean = pitchers_clean.set_index('key_retro')

In [ ]:
batters_clean = batters[
    [
        "key_retro",
        "year",
        "k_percent",
        "bb_percent",
        "exit_velocity_avg",
        "launch_angle_avg",
        "oz_contact_percent",
        "iz_contact_percent",
        "whiff_percent",
        "swing_percent",
        # "sprint_speed",
    ]
]

# Drop any batters that don't have an average exit velo or oz_contact_percent
batters_clean = batters_clean.dropna(subset=["exit_velocity_avg", "oz_contact_percent"])

# Create lookup dataframe for batter handedness from retrosheet data
plays["year"] = pd.to_datetime(plays["date"].astype(str), format='%Y%m%d').dt.year
bat_hand_df = plays[["batter", "year", "bathand"]].drop_duplicates()

def resolve_bathand(group):
    if 'B' in group.values:
        return 'B'
    else:
        return group.mode().iloc[0]  # most frequent value

bat_hand_df = (
    plays.groupby(['batter', 'year'])['bathand']
    .agg(resolve_bathand)
    .reset_index()
)

# Add to batter dataset
batters_clean = pd.merge(
    batters_clean, 
    bat_hand_df, 
    left_on=['key_retro', 'year'], 
    right_on=['batter', 'year'], 
    how='left'
)

# drop batters column (same as key_retro)
batters_clean.drop('batter', axis=1, inplace=True)

# Set the index to be the key and year
batters_clean = batters_clean.set_index("key_retro")

Cleaning play by play data

In [ ]:
# Ensure that there are no duplicate rows 
plays = plays.drop_duplicates()

# Only want to include regular season games 
plays = plays[plays["gametype"] == "regular"]

In [ ]:
# Extract the event based on retrosheet's event text 
def simplify_event(input_str):
    # Define the regular expressions for each case
    patterns = [
        (r'^[0-9]!?/G', 'ground_out'),
        (r'^[0-9]!?/L', 'line_out'),
        (r'^[0-9]!?/P|[0-9]/IF', 'pop_up'),
        (r'^[0-9]!?/F', 'fly_out'),
        (r'^[0-9]!?/BG', 'bunt_ground_out'),
        (r'^[0-9]!?/BP', 'bunt_pop_up'),
        (r'^[0-9]!?/BL', 'bunt_line_drive'), 
        (r'^[0-9]!?/SH', 'sacrifice_bunt'), 
        (r'^[0-9]!?/?/SF', 'sacrifice_fly'), 
        (r'^[0-9]!?[0-9]', 'ground_out'),
        (r'^[0-9]\([^)]*\)|^[0-9][0-9]\([^)]*\)', 'grounded_double_play'),
        (r'^[0-9]\([B]\)[0-9]\([^)]*\)', 'lined_double_play'),
        (r'^[0-9]/BINT', 'batter_interference'), 
        (r'^[C]/E[1-3]', 'interference'),
        (r'^S[0-9]|^S/|S.', 'single'),
        (r'^D[0-9]|^D/', 'double'),
        (r'^T[0-9]|^T/', 'triple'),
        (r'^DGR', 'ground_rule_double'),
        (r'^E[0-9]|^[0-9]E[0-9]', 'error'),
        (r'^FC', 'fielders_choice'),
        (r'^FLE[0-9]', 'error_foul_fly_ball'),
        (r'^HP', 'hit_by_pitch'),
        (r'^H[0-9]|^HR[0-9]', 'inside_park_home_run'),
        (r'^H|^HR', 'home_run'),
        (r'^K', 'strikeout'),
        (r'^NP', 'no_play'),
        (r'^I|^IW', 'intentional_walk'),
        (r'^W', 'walk'),
        (r'^BK', 'balk'),
        (r'^CS[2-3H]', 'caught_stealing'),
        (r'^DI', 'defensive_indifference'),
        (r'^OA', 'base_runner_advance'),
        (r'^PB', 'passed_ball'),
        (r'^WP', 'wild_pitch'),
        (r'^PO', 'picked_off'),
        (r'^SB[2-3H]', 'stolen_base'), 
        (r'^[0-9]', 'out'), 
    ]
    
    # Iterate through the patterns and return the matched outcome
    for pattern, outcome in patterns:
        if re.match(pattern, input_str):
            return outcome
    
    # If no match, return Nones
    return None

In [ ]:
plays.loc[:, "simple_event"] = plays["event"].astype(str).apply(simplify_event)

In [ ]:
# Focusing on result of the pitcher vs hitter matchup which happens where there is a plate appearance
# The other events focus on the runners
plate_app = plays[plays["pa"] == 1]

In [ ]:
len(np.unique(plate_app["simple_event"]))

In [ ]:
# From the event want to create a more general outcome variable for prediction 
def determine_outcome(event, hittype): 
    if "interference" in event: 
        return "interference"
    elif "bunt" in event: 
        return "bunt"
    elif "walk" in event: 
        return "walk"
    elif "hit_by_pitch" in event: 
        return "hitbypitch"
    elif event in ["fielders_choice", "ground_out", "grounded_double_play"]: 
        return "groundball"
    elif event in ["fly_out", "sacrifice_fly", "home_run", "ground_rule_double", "inside_park_home_run"]: 
        return "flyball"
    elif event == "line_out": 
        return "linedrive"
    elif event == "pop_up": 
        return "popup"
    elif event == "strikeout": 
        return "strikeout"
    elif event == "home_run": 
        return "flyball"
    elif event in ["single", "double", "triple", "error"]: 
        if hittype in ["BG", "BL", "BP"]: 
            return "bunt"
        elif hittype == "F": 
            return "flyball"
        elif hittype == "G": 
            return "groundball"
        elif hittype == "L": 
            return "linedrive"
        elif hittype == "P": 
            return "popup"
        else: 
            return None
    else: 
        return None
    
def simple_outcome(outcome): 
    if outcome in ["flyball", "popup", "linedrive"]: 
        return "in_play_air"
    elif outcome in ["groundball", "bunt"]: 
        return "in_play_ground"
    elif outcome in ["walk", "hitbypitch"]: 
        return "walk_hbp"
    elif outcome == "strikeout": 
        return "strikeout"
    else:
        return None
    
def super_simple_outcome(simple_outcome): 
    if simple_outcome in ["in_play_air", "in_play_ground"]: 
        return "in_play"
    elif simple_outcome in ["strikeout", "walk_hbp"]: 
        return "not_in_play"
    else:
        return None

In [ ]:
plate_app.loc[:, "outcome"] = plate_app.apply(lambda row: determine_outcome(row["simple_event"], row["hittype"]), axis=1)

In [ ]:
plate_app_clean = plate_app[
    [
        "gid",
        "outcome", 
        "inning",
        "top_bot", 
        "batter",
        "pitcher",
        "lp",
        "nump",
        "outs_pre",
        "br1_pre",
        "br2_pre",
        "br3_pre",
        "pn",
        "date",
    ]
]

# Convert runner column to 0 and 1
runner_cols = ["br1_pre", "br2_pre", "br3_pre"]
plate_app_clean.loc[:, runner_cols] = np.where(plate_app_clean[runner_cols].isna(), 0, 1)

# Convert date column to datetime
plate_app_clean.loc[:, "date"] = pd.to_datetime(plate_app_clean["date"].astype(str), format='%Y%m%d')

# Add column for appearance number in game
plate_app_clean["num_times_faced_in_game"] = plate_app_clean.groupby(['gid', 'batter', 'pitcher']).cumcount() + 1

# Drop any events with a missing outcome 
plate_app_clean.dropna(subset=["outcome"], inplace=True)

# Drop any events that were an unclassified error or interference 
plate_app_clean = plate_app_clean.loc[~plate_app_clean["outcome"].isin(["interference", "error"])]

# Add columns for simple and super simple outcome 
plate_app_clean["simple_outcome"] = plate_app_clean["outcome"].apply(simple_outcome)
plate_app_clean["super_simple_outcome"] = plate_app_clean["simple_outcome"].apply(super_simple_outcome)

# Create a column for an plate appearance id (pa_id)
plate_app_clean["pa_id"] = plate_app_clean["gid"].astype(str) + "_" + plate_app_clean["pn"].astype(str)

# Remove the gid and pn columns now that pa_id exists
plate_app_clean.drop(['gid', 'pn'], axis=1, inplace=True)

plate_app_clean = plate_app_clean.set_index(["pa_id", "batter", "pitcher"])

In [ ]:
keys, counts = np.unique(plate_app_clean["outcome"].astype(str), return_counts=True)
sorted_indices = np.argsort(counts)[::-1]
keys = keys[sorted_indices]
counts = counts[sorted_indices]

fig, ax = plt.subplots()
bars = ax.bar(keys, counts)

# Format labels with commas
ax.bar_label(bars, labels=[f"{c:,}" for c in counts])

ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))

plt.xticks(rotation=-45, ha='left')
plt.xlabel("Outcome")
plt.ylabel("Count")
plt.title("Distribution of Plate Appearance Outcomes")
plt.show()

Save Cleaned Datasets 

In [ ]:
pitchers_clean.to_csv("datasets/clean/pitchers.csv")
batters_clean.to_csv("datasets/clean/batters.csv")
plate_app_clean.to_csv("datasets/clean/plate_apps.csv")

Player Lookup

In [ ]:
def find_player_by_retro_id(retro_id): 
    player_row = register.loc[register["key_retro"] == retro_id]
    if not player_row.empty:
        return f"{player_row.iloc[0]['name_first']} {player_row.iloc[0]['name_last']}"
    else:
        return None 
    
def find_player_by_name(first_name, last_name): 
    player_row = register.loc[(register["name_first"] == first_name) & (register["name_last"] == last_name)]
    if not player_row.empty:
        return f"{player_row.iloc[0]['key_retro']}"
    else:
        return None 

In [ ]:
find_player_by_retro_id("abrac001")

In [ ]:
plays.loc[(plays["pitcher"] == "rea-c001") & (plays["batter"] == "keplm001")]

In [ ]:
np.unique(plays["bathand"].astype(str))

In [ ]:
for col in plays.columns: 
    print(col)

In [ ]:
find_player_by_name("CJ", "Abrams")

In [ ]:
register.loc[register["key_retro"] == "abrac001"]["name_first"]
#[["name_first", "name_last"]]